## Message Translator (FunctionalTransformer)

This notebook demonstrates the Message Translator pattern using a `FunctionalTransformer` with `JxScript` and `jx.assign` for structured JSONata expression building. Unlike PayloadTransformer which only transforms the payload, FunctionalTransformer can modify the entire MessageRoutable including routing information.

**Input Format**: `OrderRequest` - An e-commerce order  
**Output Format**: `InventoryCheck` - An inventory validation request

**Transformation Benefits with jx.assign:**
- **Complex Logic**: Calculates order totals and determines priority levels
- **Dynamic Routing**: Routes messages based on calculated values
- **Reusable Variables**: Intermediate calculations can be reused
- **Better Readability**: Complex transformations broken into named steps

The flow will be:

[ProbeAgent] --`OrderRequest`-> (default_topic) -> [Translator Agent/TranslatorAgent:wire_message] --`InventoryCheck`-> (inventory_high_priority | inventory_normal)

The routing destination is determined based on the order total during translation.

In [ ]:
from pydantic import BaseModel
from typing import List
from decimal import Decimal

# Input message format - e-commerce order
class OrderItem(BaseModel):
    product_id: str
    quantity: int
    unit_price: float

class OrderRequest(BaseModel):
    order_id: str
    customer_id: str
    items: List[OrderItem]
    shipping_address: str

# Output message format - inventory check
class InventoryItem(BaseModel):
    sku: str
    requested_quantity: int
    
class InventoryCheck(BaseModel):
    check_id: str
    order_reference: str
    customer_reference: str
    items_to_check: List[InventoryItem]
    total_value: float
    priority_level: str

In [ ]:
from rustic_ai.core.agents.eip.basic_wiring_agent import BasicWiringAgent
from rustic_ai.core.guild.builders import AgentBuilder, GuildBuilder, RouteBuilder
from rustic_ai.core.guild.dsl import GuildTopics
from rustic_ai.core.utils.basic_class_utils import get_qualified_class_name
from rustic_ai.core.utils import jx
from rustic_ai.core.utils.jexpr import JObj, JxScript
import uuid

# Create the translator agent
translator_agent = (
    AgentBuilder(BasicWiringAgent)
    .set_id("TranslatorAgent")
    .set_name("Message Translator")
    .set_description("Translates order requests into inventory checks using FunctionalTransformer with dynamic routing")
    .build_spec()
)



In [ ]:

# Create a JSONata transformation script using JxScript with jx.assign
# This transforms the payload AND determines dynamic routing
transformation_script = JxScript(
    # Assign intermediate variables for better readability
    jx.assign("$timestamp", jx.millis()),
    jx.assign("$random_value", jx.JExpr("$random()")),
    jx.assign("$random_suffix", jx.substring(jx.string("$random_value"), 2, 6)),
    jx.assign("$timestamp_str", jx.string("$timestamp")),
    jx.assign("$check_prefix", jx.substringBefore("$timestamp_str", ".")),
    jx.assign("$generated_check_id", jx.JExpr('"INV-" & $check_prefix & "-" & $random_suffix')),
    
    # Calculate order total for routing decisions
    jx.assign("$order_total", jx.JExpr("$sum(payload.items.(quantity * unit_price))")),
    jx.assign("$is_high_priority", jx.JExpr("$order_total > 1000")),
    jx.assign("$priority_level", jx.ternary("$is_high_priority", "high", "normal")),
    jx.assign("$routing_topic", jx.ternary("$is_high_priority", "inventory_high_priority", "inventory_normal")),
    
    # Transform items to inventory check format
    jx.assign("$inventory_items", jx.JExpr("""
        payload.items.{
            "sku": product_id,
            "requested_quantity": quantity
        }
    """)),
    
    # Build the transformed payload
    jx.assign("$transformed_payload", JObj({
        "check_id": jx.JExpr("$generated_check_id"),
        "order_reference": jx.JExpr("payload.order_id"),
        "customer_reference": jx.JExpr("payload.customer_id"), 
        "items_to_check": jx.JExpr("$inventory_items"),
        "total_value": jx.JExpr("$order_total"),
        "priority_level": jx.JExpr("$priority_level")
    })),
    
    # Final MessageRoutable structure with dynamic routing
    JObj({
        "payload": jx.JExpr("$transformed_payload"),
        "topics": jx.JExpr("$routing_topic"),
        "format": get_qualified_class_name(InventoryCheck)
    })
)


In [ ]:
# Create the routing rule with FunctionalTransformer
translation_rule = (
    RouteBuilder(translator_agent)
    .filter_on_origin(origin_message_format=get_qualified_class_name(OrderRequest))
    .set_functional_transformer(transformation_script)
    .build()
)


In [ ]:
# Create the guild
guild = (
    GuildBuilder(
        guild_id="MessageTranslatorFunctionalGuild",
        guild_name="Message Translator (Functional) Guild",
        guild_description="A guild demonstrating message translation using FunctionalTransformer with dynamic routing.",
    )
    .add_agent_spec(translator_agent)
    .add_route(translation_rule)
    .launch("myorg")
)

In [ ]:
from rustic_ai.core.agents.testutils.probe_agent import ProbeAgent

guild_default_topic = GuildTopics.DEFAULT_TOPICS[0]

probe_spec = (
    AgentBuilder(ProbeAgent)
    .set_id("ProbeAgent")
    .set_name("Probe Agent")
    .set_description("A probe agent to test message translation with dynamic routing.")
    .add_additional_topic("inventory_high_priority")
    .add_additional_topic("inventory_normal")
    .build_spec()
)

probe_agent: ProbeAgent = guild._add_local_agent(probe_spec)  # type: ignore

In [ ]:
# Send a high-value order (should route to inventory_high_priority)
high_value_order = OrderRequest(
    order_id="ORD-001",
    customer_id="CUST-12345",
    items=[
        OrderItem(product_id="LAPTOP-001", quantity=2, unit_price=899.99),  # $1799.98
        OrderItem(product_id="MOUSE-001", quantity=2, unit_price=49.99)     # $99.98
    ],  # Total: $1899.96 (> $1000, should be high priority)
    shipping_address="123 Main St, City, State 12345"
)

# Calculate total for demonstration
total_value = sum(item.quantity * item.unit_price for item in high_value_order.items)
print(f"High-value order total: ${total_value:.2f} (should route to high priority)")

probe_agent.publish_with_guild_route(
    payload=high_value_order,
    topic=guild_default_topic,
)

In [ ]:
# Check the translated message and routing
probe_agent.print_message_history()

In [ ]:
# Send a low-value order (should route to inventory_normal)
low_value_order = OrderRequest(
    order_id="ORD-002", 
    customer_id="CUST-67890",
    items=[
        OrderItem(product_id="BOOK-001", quantity=3, unit_price=15.99),    # $47.97
        OrderItem(product_id="PEN-001", quantity=5, unit_price=2.50)       # $12.50
    ],  # Total: $60.47 (< $1000, should be normal priority)
    shipping_address="456 Oak Ave, Town, State 67890"
)

# Calculate total for demonstration
total_value = sum(item.quantity * item.unit_price for item in low_value_order.items)
print(f"Low-value order total: ${total_value:.2f} (should route to normal priority)")

probe_agent.publish_with_guild_route(
    payload=low_value_order,
    topic=guild_default_topic,
)

In [ ]:
# Check all messages to see both translations and different routing
probe_agent.print_message_history()

In [ ]:
# Inspect the last two messages to see the transformation and routing
messages = probe_agent.get_messages()
print("Message Analysis:")
print("=" * 50)

for i, msg in enumerate(messages[-2:], 1):
    print(f"\nMessage {i}:")
    print(f"  Format: {msg.format}")
    print(f"  Topic: {msg.topic_published_to}")
    print(f"  Payload Type: {type(msg.payload).__name__}")
    if hasattr(msg.payload, 'total_value'):
        print(f"  Total Value: ${msg.payload.total_value}")
        print(f"  Priority Level: {msg.payload.priority_level}")
        print(f"  Check ID: {msg.payload.check_id}")
    print(f"  Routing Decision: {'High Priority' if 'high_priority' in msg.topic_published_to else 'Normal Priority'}")